# Инструменты для работы с языком 

## Домашнее задание

все материалы для выполения дз в `sem2.ipynb`


### Задание 1.

**Задание**: обучите три классификатора: 

1) на токенах с высокой частотой 

2) на токенах со средней частотой 

3) на токенах с низкой частотой


Сравните полученные результаты, оцените какие токены наиболее важные для классификации.


### Задание 2.

найти фичи с наибольшей значимостью, и вывести их


### Задание 3.

1) сравнить count/tf-idf/hashing векторайзеры/полносвязанную сетку (построить classification_report)

2) подобрать оптимальный размер для hashing векторайзера 

3) убедиться что для сетки нет переобучения

In [1]:
# если у вас линукс / мак / collab или ещё какая-то среда, в которой работает wget, можно так:
!wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
!wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv

--2022-10-13 12:36:05--  https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
Распознаётся www.dropbox.com (www.dropbox.com)… 162.125.70.18
Подключение к www.dropbox.com (www.dropbox.com)|162.125.70.18|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 302 Found
Адрес: /s/raw/fnpq3z4bcnoktiv/positive.csv [переход]
--2022-10-13 12:36:05--  https://www.dropbox.com/s/raw/fnpq3z4bcnoktiv/positive.csv
Повторное использование соединения с www.dropbox.com:443.
HTTP-запрос отправлен. Ожидание ответа… 302 Found
Адрес: https://uc6ab235df79b91cfce871b25811.dl.dropboxusercontent.com/cd/0/inline/BuvBLbKZuG5u5y5ewNxYzCTCX64RH2RF3ftwyViUg0jT9dx1ZMnTrUFoPu7I6bUPWrA1Ow6n65UIpPME2ZZobZLNxNYoKpHklySK0zjiaEAMfL2mDG22tDhjTYtGk9qTymJ9tWy7Tm9ihnEl5B2QZvCOwI-iNl_tNr2gPCucylhQHQ/file# [переход]
--2022-10-13 12:36:06--  https://uc6ab235df79b91cfce871b25811.dl.dropboxusercontent.com/cd/0/inline/BuvBLbKZuG5u5y5ewNxYzCTCX64RH2RF3ftwyViUg0jT9dx1ZMnTrUFoPu7I6bUPWrA1Ow6n65UIpPME2ZZobZLNxNYoKpHk

In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [3]:
# считываем данные и заполняем общий датасет
positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['text'])
positive['label'] = ['positive'] * len(positive)
negative = pd.read_csv('negative.csv', sep=';', usecols=[3], names=['text'])
negative['label'] = ['negative'] * len(negative)
df = pd.concat([positive,negative])

In [4]:
df.head()

,text,label
0,"@first_timee хоть я и школота, но поверь, у на...",positive
1,"Да, все-таки он немного похож на него. Но мой ...",positive
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,positive
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",positive
4,@irina_dyshkant Вот что значит страшилка :D\nН...,positive


In [5]:
df.tail()

,text,label
111918,Но не каждый хочет что то исправлять:( http://...,negative
111919,скучаю так :-( только @taaannyaaa вправляет мо...,negative
111920,"Вот и в школу, в говно это идти уже надо(",negative
111921,"RT @_Them__: @LisaBeroud Тауриэль, не грусти :...",negative
111922,Такси везет меня на работу. Раздумываю приплат...,negative


In [6]:
x_train, x_test, y_train, y_test = train_test_split(df.text, df.label)

In [7]:
from sklearn.linear_model import LogisticRegression 
from sklearn.feature_extraction.text import CountVectorizer

from nltk import ngrams
import nltk
from nltk import collocations 
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk.corpus import stopwords

from collections import Counter

In [8]:
nltk.download('genesis')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package genesis to /Users/dv/nltk_data...
[nltk_data]   Package genesis is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/dv/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/dv/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [9]:
from string import punctuation
noise = stopwords.words('russian') + list(punctuation)

In [10]:
corpus = [token for tweet in df.text for token in word_tokenize(tweet) if token not in punctuation]
print(len(corpus))
corpus[:10]

2870536


['first_timee', 'хоть', 'я', 'и', 'школота', 'но', 'поверь', 'у', 'нас', 'то']

In [11]:
freq_dict = Counter(corpus)
freq_dict_sorted= sorted(freq_dict.items(), key=lambda x: -x[1])
list(freq_dict_sorted)[:10]

[('не', 69472),
 ('и', 55166),
 ('в', 52902),
 ('я', 52818),
 ('RT', 38070),
 ('на', 35759),
 ('http', 32998),
 ('что', 31541),
 ('с', 27217),
 ('а', 26860)]

## Задание 1.

**Задание**: обучите три классификатора: 

1) на токенах с высокой частотой 

2) на токенах со средней частотой 

3) на токенах с низкой частотой

Сравните полученные результаты, оцените какие токены наиболее важные для классификации.

In [12]:
high_tokens = set()
medium_tokens = set()
low_tokens = set()

ht_value = 5000
mt_value = 1000

for i in freq_dict_sorted:
    if i[1] > ht_value:
        high_tokens.add(i[0])
    elif i[1] < mt_value:
        low_tokens.add(i[0])
    else:
        medium_tokens.add(i[0])

print(len(high_tokens),len(medium_tokens),len(low_tokens))

56 199 350868


In [13]:
import warnings
warnings.filterwarnings("ignore")

In [14]:
def MyFit_Report (stop_words_list):

    vec = CountVectorizer(ngram_range=(1, 1), tokenizer=word_tokenize, stop_words=stop_words_list)
    bow = vec.fit_transform(x_train)
    clf = LogisticRegression(max_iter=10000, random_state=42)
    clf.fit(bow, y_train)
    pred = clf.predict(vec.transform(x_test))

    print(classification_report(pred, y_test))

In [15]:
# токены с высокой частотой
MyFit_Report(noise + list(low_tokens) + list(medium_tokens))

              precision    recall  f1-score   support

    negative       0.78      0.59      0.67     37448
    positive       0.46      0.68      0.55     19261

    accuracy                           0.62     56709
   macro avg       0.62      0.63      0.61     56709
weighted avg       0.67      0.62      0.63     56709



In [16]:
# токены со средней частотой
MyFit_Report(noise + list(high_tokens) + list(low_tokens))

              precision    recall  f1-score   support

    negative       0.44      0.66      0.53     18560
    positive       0.78      0.59      0.67     38149

    accuracy                           0.61     56709
   macro avg       0.61      0.63      0.60     56709
weighted avg       0.67      0.61      0.62     56709



In [17]:
# токены с низкой частотой
MyFit_Report(noise + list(high_tokens) + list(medium_tokens))

              precision    recall  f1-score   support

    negative       0.80      0.74      0.77     30243
    positive       0.73      0.78      0.75     26466

    accuracy                           0.76     56709
   macro avg       0.76      0.76      0.76     56709
weighted avg       0.76      0.76      0.76     56709



Лучшая точность при выставленных границах получается у низкочастотных токенов. При изменении критериев деления по частоте точность меняется

## Задание 2.

найти фичи с наибольшей значимостью, и вывести их

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [19]:
vec = TfidfVectorizer(ngram_range=(1, 1), tokenizer=word_tokenize, stop_words=stopwords.words('russian'))
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       1.00      1.00      1.00     28124
    positive       1.00      1.00      1.00     28585

    accuracy                           1.00     56709
   macro avg       1.00      1.00      1.00     56709
weighted avg       1.00      1.00      1.00     56709



In [20]:
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [21]:
tokens_arr = []
for i in punctuation:
    cool_token = i
    pred = ['positive' if cool_token in tweet else 'negative' for tweet in x_test]
    tokens_arr.append((i,accuracy_score(pred, y_test)))
print(tokens_arr)

[('!', 0.5190005113826729), ('"', 0.5089315628912519), ('#', 0.506744961117283), ('$', 0.49688761924914915), ('%', 0.49888024828510463), ('&', 0.49651730765839636), ("'", 0.4967289142816837), ('(', 0.025057750974272163), (')', 0.9131883827963815), ('*', 0.5152621277045971), ('+', 0.4969757886755189), (',', 0.5083849124477596), ('-', 0.510113033204606), ('.', 0.5107302191891939), ('/', 0.546544640180571), (':', 0.5466680773774886), (';', 0.5007318062388686), ('<', 0.4967289142816837), ('=', 0.49651730765839636), ('>', 0.4967289142816837), ('?', 0.5065686222645436), ('@', 0.5687280678551906), ('[', 0.49676418205223155), ('\\', 0.49676418205223155), (']', 0.4968699853638752), ('^', 0.503359255144686), ('_', 0.5188065386446595), ('`', 0.4963586026909309), ('{', 0.4967289142816837), ('|', 0.4925673173570333), ('}', 0.4967465481669576), ('~', 0.4968523514786013)]


Фичи с наибольшей значимостью

In [22]:
for i in sorted(tokens_arr, key=lambda x: -x[1])[:10]:
    print(i[0],'  -   ' ,i[1])

)   -    0.9131883827963815
@   -    0.5687280678551906
:   -    0.5466680773774886
/   -    0.546544640180571
!   -    0.5190005113826729
_   -    0.5188065386446595
*   -    0.5152621277045971
.   -    0.5107302191891939
-   -    0.510113033204606
"   -    0.5089315628912519


### Задание 3.

1) сравнить count/tf-idf/hashing векторайзеры/полносвязанную сетку (построить classification_report)

2) подобрать оптимальный размер для hashing векторайзера 

3) убедиться что для сетки нет переобучения

In [23]:
count_vect = CountVectorizer(ngram_range=(1, 1), tokenizer=word_tokenize, stop_words=stopwords.words('russian'))
bow = count_vect.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       1.00      0.63      0.78     44361
    positive       0.43      1.00      0.60     12348

    accuracy                           0.71     56709
   macro avg       0.72      0.82      0.69     56709
weighted avg       0.88      0.71      0.74     56709



In [24]:
count_vect = CountVectorizer(ngram_range=(1, 1), tokenizer=word_tokenize, stop_words=stopwords.words('russian'))
bow = count_vect.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       1.00      0.63      0.78     44361
    positive       0.43      1.00      0.60     12348

    accuracy                           0.71     56709
   macro avg       0.72      0.82      0.69     56709
weighted avg       0.88      0.71      0.74     56709



In [25]:
from sklearn.feature_extraction.text import HashingVectorizer

In [26]:
n_features = [10, 100, 1000, 10000, 100000, 200000, 300000, 400000]
for n in n_features:
    vectorizer = HashingVectorizer(n_features=n,)
    bow = vectorizer.fit_transform(x_train)
    clf = LogisticRegression(random_state=42)
    clf.fit(bow, y_train)
    pred = clf.predict(vectorizer.transform(x_test))
    print(f'n_features = {n}')
    print(classification_report(pred, y_test))
    print(10*'=')

n_features = 10
              precision    recall  f1-score   support

    negative       0.49      0.54      0.52     25377
    positive       0.60      0.54      0.57     31332

    accuracy                           0.54     56709
   macro avg       0.54      0.54      0.54     56709
weighted avg       0.55      0.54      0.54     56709

n_features = 100
              precision    recall  f1-score   support

    negative       0.57      0.59      0.58     26978
    positive       0.62      0.59      0.60     29731

    accuracy                           0.59     56709
   macro avg       0.59      0.59      0.59     56709
weighted avg       0.59      0.59      0.59     56709

n_features = 1000
              precision    recall  f1-score   support

    negative       0.63      0.65      0.64     27185
    positive       0.67      0.65      0.66     29524

    accuracy                           0.65     56709
   macro avg       0.65      0.65      0.65     56709
weighted avg       0.65

Переобучение не возникает, оптимальное n_features = 200000